# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

### 1. 环境准备与库导入

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/deepseek-quickstart/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [ ]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### 3. 微调前推理测试

In [6]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [7]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [8]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [9]:
print(response[0].split("### Response:")[1])


<think>
好的，我需要分析这位28岁的男性，他是一名程序员，最近一周每天工作到半夜，出现头晕、脖子疼和恶心等症状。首先，头晕和脖子疼可能是脑力运动导致的，但持续的时间和频率可能提示有神经系统疾病，如脑膜炎、中风或脑膜炎。恶心可能是胃酸过多引起的胃食管反流症状。考虑到他工作时间长期，可能需要休息或调整工作方式。

接下来，我需要考虑是否有其他可能的解释，比如中风或脑膜炎。中风可能需要立即就医，但如果是长期加班导致的持续性疼痛，可能需要观察和休息。恶心可能与胃酸过多有关，可能需要胃药治疗，但需要在医生指导下进行。

此外，我需要考虑是否有其他因素，比如睡眠质量。长时间工作可能导致睡眠不足，影响神经功能。如果他最近有睡眠问题，可能需要改善睡眠习惯，比如早睡早起，避免过度劳累。

综合来看，建议他立即就医，以排除可能的中风或脑膜炎。同时，观察是否有其他症状，如恶心可能需要进一步检查。建议他寻求专业医生的诊断和治疗，以确保他的健康状况得到妥善处理。
</think>

首先，根据患者的描述，头晕、脖子疼和恶心可能是由中风、脑膜炎或其他神经系统疾病引起的。考虑到他的工作长期性和持续性疼痛，建议立即就医，以排除可能的中风或脑膜炎。如果出现胃酸过多导致的恶心，可能需要胃药治疗，但需在医生指导下进行。建议他尽快与医生沟通，观察是否有其他症状，如恶心，以及是否有其他健康问题。建议他寻求专业医生的诊断和治疗，以确保他的健康状况得到妥善处理。


### 4. 下载和格式化训练数据集

医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

In [10]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [11]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [12]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [13]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

In [14]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

In [15]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Detected kernel version 5.4.119, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### 7. 开始训练

In [16]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 57 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (57 x 2 x 1) = 114
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 51 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 25 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (25 x 2 x 1) = 50
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Step,Training Loss
1,3.214200
2,3.143100
3,3.106300
4,3.185300
5,3.071100
6,2.977400
7,2.931600
8,2.794300
9,2.852200
10,2.851900


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 22 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (22 x 2 x 1) = 44
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Step,Training Loss
1,2.329000
2,2.284400
3,2.253000
4,2.257000
5,2.370900
6,2.256200
7,2.144100
8,2.262300
9,2.135400
10,2.219600


TrainOutput(global_step=60, training_loss=2.2135083158810933, metrics={'train_runtime': 433.3618, 'train_samples_per_second': 6.092, 'train_steps_per_second': 0.138, 'total_flos': 2.2651084149264384e+16, 'train_loss': 2.2135083158810933})


### 8. 保存微调后的模型（Lora）

In [17]:
model.save_pretrained("qwen-1.5b_lora_model")

In [18]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [19]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [21]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这个病人已经5天了，急性阑尾炎，还有一点发热。嗯，看到这个病人，我先想到的是应该处理他的症状，比如让他的疼痛减轻，同时看看是否能让他不要发热。

嗯，病人有压痛包块，这可能是阑尾炎导致的，因为压痛包块通常和阑尾炎有关。而且，他发热了，可能需要一些药物来缓解发热，比如对乙酰氨基酚。

不过，我得想想，如果他有压痛包块，可能会导致疼痛加重，甚至让他的身体失去平衡。这样，药物可能需要更谨慎地使用，不能随便用。

嗯，想想看，如果病人有压痛包块，最好先用一些止痛药，比如左旋曲苷，这样能减少疼痛。同时，要小心不要让他的疼痛太大，以免加重压痛包块。

然后，再想想，他发热了，可能需要一些退火药物，比如对乙酰氨基酚。不过，要小心不能用得太快，否则会刺激疼痛，甚至引起药物反应。

嗯，看来这个病人应该先用左旋曲苷来缓解疼痛，然后慢慢加药，同时注意药物的使用，防止影响到他的疼痛和发热。

所以，我觉得对乙酰氨基酚应该先加慢一点，这样可以避免药物的副作用，同时确保疼痛减轻和发热的控制。

嗯，现在看来，这个病人应该先加慢对乙酰氨基酚，再慢慢增加，同时使用左旋曲苷来缓解疼痛，这样可以更安全地处理他的症状。这样，他应该能更好地应对这些症状。
</think>
对于一个已经发病5天的急性阑尾炎患者，出现右下腹有压痛包块且发热的情况，建议采取以下步骤：

1. **评估疼痛和发热情况**：首先确认压痛包块是否合理，并评估疼痛和发热的严重程度。这些症状可能需要药物的适当缓解。

2. **选择药物**：使用左旋曲苷作为止痛药，可以减轻疼痛，但需在使用前谨慎，因为这可能引起疼痛加重。

3. **缓慢用药**：对乙酰氨基酚的使用要缓慢，以避免刺激疼痛，并控制其影响。通常在医生指导下使用，避免超量使用。

4. **同时控制发热**：在对乙酰氨基酚的使用过程中，应小心控制其剂量，避免导致发热增加或药物反应。可使用退火药如对乙酰氨基酚，但需根据患者的情况调整用药量。

5. **观察症状变化**：在开始使用药物后，持续观察患者症状变化，必要时调整用药方案，以确保疼痛减轻和发热的稳定。

6. **医疗建议**：在医生的指导下，根据患者的具体情况调整用药方案，并在患者病情稳定后再进行更全面的治疗。

建议在患者症状持续时，逐步调整用药，同时密切观察症状变化，以确保药物的使用安全性和有效

In [22]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [23]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这个60岁的患者，他的右侧胸疼让我很困扰。X线检查显示右侧肋膈角消失，这让我想到了肺结核，但胸腔里有积液，这似乎有点不对劲。通常，肺结核会有胸腔积液，但具体性质呢？嗯，可能是结核病，也可能有其他原因。

我们得弄清楚这个积液到底是什么类型的。如果胸腔积液是结核病引起的，那它应该属于结核性胸腔积液。可是，如果是其他原因导致的，比如肺结核可能也会有其他表现。嗯，所以可能需要进一步的检验来确认。

我记得在诊断结核的时候，通常会进行胸腔的活检，看胸腔里有没有结核性胸腔积液。如果结果是肯定的，那我们就可以明确诊断了。所以，如果胸腔积液是结核性的话，活检是个好选择。但是，如果不确定是不是结核性的话，可能会选择其他方法，比如结核病的辅助检查。

所以，如果胸腔积液是结核病，活检是个不错的选择。如果不确定，可能需要检查其他因素，比如结核病的辅助检查。这让我想到，这可能涉及到对患者进行更全面的检查，以确保我们能准确地诊断出原因。

哦，对了，结核病的辅助检查包括结核病的胸腔活检和结核病的体格检查。如果要确认是结核性胸腔积液，结核病的胸腔活检是关键。如果不确定，可能需要结合其他检查结果，比如结核病的体格检查，来全面了解患者的情况。

所以，最终，如果确定是肺结核，那么结核病的胸腔活检是个好方法。如果不确定，可能需要结合其他辅助检查，以确保我们能准确地诊断出病因。这让我觉得，选择合适的实验室检查非常重要，以帮助我们快速定位和诊断出患者的病因。
</think>
对于该60岁男性患者的情况，首先需要明确胸腔积液的性质。如果胸腔积液是结核性，那么选择结核病的胸腔活检是个有效的方法，以确定诊断。如果不确定是结核性胸腔积液，通常会结合结核病的辅助检查，如结核病的体格检查，以全面了解患者病因。因此，针对该患者的情况，建议选择结核病的胸腔活检来确定胸腔积液的性质。


In [24]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位28岁的男性患者最近工作是程序员，而且经常熬夜，这让我想到，可能是神经衰竭的迹象。因为经常熬夜可能会导致神经的疲劳，从而引发一些神经系统的问题。

但是，现在他又感觉头晕目眩，甚至有点恶心，这让我想到更复杂的情况。头晕目眩，恶心，这些症状都可能和脑力活动有关，尤其是长时间的神经刺激。

再想想，脑力活动是程序员的主要工作，长时间的神经刺激就可能让脑供血不足，从而导致脑供血不足，进而引发脑供血不足导致的脑卒中。脑卒中的症状包括晕厥、恶心、头晕，这些都与脑供血不足有关。

所以，结合他的工作和症状，脑卒中的可能性比较高。脑卒中的病因主要是脑供血不足，而长时间的脑力活动和神经刺激就是导致供血不足的原因。

所以，综合来看，这位28岁男性患者的情况很可能是脑卒中的表现。再仔细想想，脑卒中的病因确实跟脑供血不足有关，而这种症状也符合他的工作和症状表现。因此，这应该是脑卒中的一个可能的原因。
</think>
这位28岁的男性患者最近工作是程序员，长期熬夜，症状包括头晕目眩和恶心，这提示可能有脑卒中的症状。脑卒中的病因主要是脑供血不足，而这种症状也与脑力活动和神经刺激有关。由于患者长期的脑力活动，脑供血不足成为可能的原因，因此，这可能是脑卒中的表现。


### 10.加载微调前模型进行对比

In [ ]:
# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model_sft_before, tokenizer_sft_before = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [28]:
# 对比
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model_sft_before, tokenizer_sft_before, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，我现在需要回答一个关于肺结核患者胸腔积液的问题。首先，题目给出的是一个60岁男性，出现右侧胸疼，X线检查显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液。现在要确定胸腔积液的性质，需要什么实验室检查。

首先，胸腔积液通常由肺结核引起，尤其是 Secondary Lungs Abscess，也就是肺结核伴胸腔积液。这种积液可能由细菌感染引起，也可能由真菌或病毒（如肺炎链球菌）引起。所以，我需要考虑不同类型的积液可能有哪些指标。

根据我所学，胸腔积液的性质可以通过以下实验室检查来判断：

1. **血钙血症检查**：血钙血症通常在真菌感染（如念珠菌）或细菌感染（如肺炎链球菌）的胸腔积液中出现，而肺结核通常不会导致血钙血症。因此，血钙血症检查不太可能帮助诊断。

2. **血尿检查**：血尿可能发生在肺结核患者中，尤其是感染细菌的胸腔积液。但如果是真菌感染的胸腔积液，通常不会出现血尿。因此，血尿检查也可能不太适用。

3. **血钙血症检查**：如前所述，这可能与细菌感染相关，而肺结核通常不带有血钙血症。

4. **胸腔镜检查**：这是标准的检查方法，用于观察积液的形状和大小，但不是诊断工具。

5. **血钙血症检查**：同样，这可能与细菌感染相关，不适用。

6. **白细胞计数**：白细胞计数可能与细菌感染相关，但肺结核通常不带有此特征。

7. **血钙血症检查**：同样，这可能与细菌感染相关，不适用。

8. **血钙血症检查**：这可能与细菌感染相关，不适用。

9. **血钙血症检查**：同样，这可能与细菌感染相关，不适用。

10. **血钙血症检查**：这可能与细菌感染相关，不适用。

11. **血钙血症检查**：这可能与细菌感染相关，不适用。

12. **血钙血症检查**：这可能与细菌感染相关，不适用。

13. **血钙血症检查**：这可能与细菌感染相关，不适用。

14. **血钙血症检查**：这可能与细菌感染相关，不适用。

15. **血钙血症检查**：这可能与细菌感染相关，不适用。

16. **血钙血症检查**：这可能与细菌感染相关，不适用。

17. **血钙血症检查**：这可能与细菌感染相关，不适用。

18. **

In [29]:
# 对比
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model_sft_before, tokenizer_sft_before, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
好，我现在要回答这个医学问题。患者是28岁男性，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。首先，我需要分析患者的症状和背景。

患者长时间熬夜，说明可能有睡眠障碍，也可能有长期的疲劳或睡眠不足。头晕目眩和恶心，可能与睡眠不足有关，也可能有其他原因，比如压力、焦虑、精神疾病等。

常见的症状包括头晕目眩、恶心呕吐，可能与长期的疲劳或睡眠问题有关。患者有长期熬夜的习惯，可能影响睡眠，导致持续的头晕目眩和恶心。

考虑到这些因素，可能的疾病包括：

1. 呼吸系统疾病：如鼻炎、多囊腺炎、中耳炎、支气管炎等。
2. 心血管疾病：如高血压、高血糖、糖尿病、心脑血管疾病。
3. 内脏疾病：如胃炎、胰腺炎、肝胆疾病。
4. 软件故障：可能是电脑故障，导致头晕目眩。
5. 心理因素：长期的压力或焦虑可能导致这些症状。

需要考虑这些可能性，可能需要进一步的诊断，比如通过影像学检查（如CT、MRI）来评估是否有睡眠障碍或其他潜在的疾病。如果怀疑是电脑故障，可以建议检查电脑的磁盘或电源，看看是否是软件问题。

总结来说，患者可能有睡眠问题，导致持续的头晕目眩，或者有长期的疲劳或压力因素，可能与心脑血管疾病有关。建议进一步的检查来明确诊断。
</think>

对于这名28岁的男性患者，工作是程序员，长期熬夜，最近突然头晕目眩，甚至恶心，可能的疾病包括以下几种：

1. **睡眠障碍**：长期熬夜可能影响睡眠，导致持续的头晕目眩和恶心。
2. **神经系统疾病**：如多囊腺炎、中耳炎、支气管炎、鼻炎等。
3. **心脑血管疾病**：如高血压、糖尿病、心脏病等。
4. **心理因素**：长期的压力或焦虑可能引发这些症状。
5. **电脑故障**：可能是电脑故障导致的头晕目眩。

建议进一步的检查，如影像学检查（如CT、MRI）来评估是否有睡眠问题或潜在的疾病。如果怀疑是电脑故障，可以建议检查电脑的磁盘或电源，看看是否是软件问题。
